# **Final Practice — Clickbait Detection**

## **Alberto — Preprocesamiento, Carga de Datos y División del Corpus**

**Equipo:**  
**Alumnos:** Celso Alberto Gallegos Sánchez,  
Montero Marin Andrea Jaqueline,  
Liceaga Cardoso Ángel David,  
Ramírez Verde Enrique  
**Materia:** Procesamiento de Lenguaje Natural  
**Fecha:** Noviembre 2025

### **1. Carga de datos**

En esta celda se realiza la carga de los corpora necesarios para la tarea de detección de clickbait:

- `TA1C_dataset_detection_train.csv`: corpus de entrenamiento original.
- `TA1C_dataset_detection_test.csv`: corpus de prueba sobre el que se generarán predicciones al final del proyecto.

Acciones principales:
- Importar las librerías necesarias (`pandas`, `pathlib`).
- Definir las rutas de los archivos.
- Cargar los CSV en `DataFrame` de `pandas`.
- Mostrar información básica de las dimensiones del corpus de entrenamiento.


In [11]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

In [9]:
DATA_DIR = Path(
    r"C:\Users\Miner\OneDrive\Documentos\7to semestre\Procesamiento de Lenguaje Natural\Final practice - Clickbait detection\Final-practice---Clickbait-detection\Material de apoyo"
)

# Rutas de los archivos
train_path = DATA_DIR / "TA1C_dataset_detection_train.csv"
test_path  = DATA_DIR / "TA1C_dataset_detection_dev.csv"   # usa 'dev' como segundo corpus

# Carga de los corpora
df_train_raw = pd.read_csv(train_path)
df_test_raw  = pd.read_csv(test_path)

print("Tamaño del corpus de entrenamiento (raw):", df_train_raw.shape)
print("Tamaño del segundo corpus (dev/test) raw:", df_test_raw.shape)

# Vista rápida de las primeras filas del train
df_train_raw.head()

Tamaño del corpus de entrenamiento (raw): (2800, 6)
Tamaño del segundo corpus (dev/test) raw: (700, 5)


,Tweet ID,Tweet Date,Media Name,Media Origin,Teaser Text,Tag Value
0,1302968016477589504,07-09-2020,El País,Uruguay,#SegundaDivisión | La fortaleza del ataque: R...,No
1,1296805148950515713,21-08-2020,El País,Uruguay,Jorge Lanata a los argentinos que se van a Uru...,No
2,1303065732884967426,07-09-2020,El País,Uruguay,Raffo: “Los montevideanos deben estar alerta p...,No
3,1302462441520074757,06-09-2020,El País,Uruguay,Ecos del universo: joven uruguayo desentraña (...,Clickbait
4,1301763104435589120,04-09-2020,El País,Uruguay,"Propuesta quinquenal de ANEP: aumento de 3,8% ...",No


### **2. Selección de columnas y preprocesamiento mínimo**

En este paso se seleccionan únicamente las columnas relevantes para la tarea:

- **`Teaser Text`** → variable de entrada (X).
- **`Tag Value`** → etiqueta/clase objetivo (y).

Acciones principales:
- Filtrar el `DataFrame` de entrenamiento para quedarnos con estas columnas.
- Eliminar filas con valores nulos en estas columnas (si existieran).
- Normalizar ligeramente el texto (por ejemplo, eliminando espacios en blanco al inicio y al final).

El resultado será un `DataFrame` limpio (`df_all`) listo para hacer la división en *train* y *dev*.


In [10]:
# Nombres de las columnas a utilizar
feature_col = "Teaser Text"
target_col  = "Tag Value"

# Nos quedamos solo con las columnas relevantes del train
df_all = df_train_raw[[feature_col, target_col]].copy()

# Eliminamos filas con valores nulos en texto o etiqueta
df_all = df_all.dropna(subset=[feature_col, target_col])

print("Tamaño del corpus de entrenamiento limpio:", df_all.shape)
df_all.head()


Tamaño del corpus de entrenamiento limpio: (2800, 2)


,Teaser Text,Tag Value
0,#SegundaDivisión | La fortaleza del ataque: R...,No
1,Jorge Lanata a los argentinos que se van a Uru...,No
2,Raffo: “Los montevideanos deben estar alerta p...,No
3,Ecos del universo: joven uruguayo desentraña (...,Clickbait
4,"Propuesta quinquenal de ANEP: aumento de 3,8% ...",No


### **3. División del corpus en train/dev (75%–25%)**

En esta celda se realiza la división del corpus de entrenamiento en dos subconjuntos:

- **Train set (75%)**: usado para entrenar el modelo.
- **Dev set (25%)**: usado para validación y ajuste de hiperparámetros.

Parámetros solicitados:
- `shuffle = True`  → mezclar las instancias antes de dividir.
- `random_state = 0` → semilla fija para reproducibilidad.
- `stratify = y` → mantener la misma proporción de clases en *train* y *dev*.

Se generan:
- `X_train`, `y_train`
- `X_dev`, `y_dev`
- `df_train` y `df_dev` como `DataFrame` finales con las columnas `Teaser Text` y `Tag Value`,
  que servirán como **dataframes finales para el entrenamiento**.

Opcionalmente, también se guardan en CSV los subconjuntos ya divididos.


In [ ]:
# Variables de entrada (X) y salida (y)
X = df_all[feature_col]
y = df_all[target_col]

# División estratificada 75% / 25%
X_train, X_dev, y_train, y_dev = train_test_split(
    X,
    y,
    test_size=0.25,
    shuffle=True,
    random_state=0,
    stratify=y
)

# Conjutos de datos finales mas faciles de manejar
df_train = pd.DataFrame({
    feature_col: X_train,
    target_col: y_train
}).reset_index(drop=True)

df_dev = pd.DataFrame({
    feature_col: X_dev,
    target_col: y_dev
}).reset_index(drop=True)

print("Tamaño df_train:", df_train.shape)
print("Tamaño df_dev:  ", df_dev.shape)

print("\nDistribución de clases en df_train:")
print(df_train[target_col].value_counts(normalize=True))

print("\nDistribución de clases en df_dev:")
print(df_dev[target_col].value_counts(normalize=True))

# Vista rápida
df_train.head()

Tamaño df_train: (2100, 2)
Tamaño df_dev:   (700, 2)

Distribución de clases en df_train:
Tag Value
No           0.715238
Clickbait    0.284762
Name: proportion, dtype: float64

Distribución de clases en df_dev:
Tag Value
No           0.714286
Clickbait    0.285714
Name: proportion, dtype: float64


,Teaser Text,Tag Value
0,La letra que ha dejado sin bote a Pablo Díaz e...,Clickbait
1,Sanidad revende 30.000 dosis de la vacuna cont...,No
2,Video. Putin presume nuevo armamento nuclear r...,No
3,"Salvador Cienfuegos, el general que nadie se a...",No
4,RT @QuePasaCL: Conocimiento mundial: ¿Cuál es ...,Clickbait


### **4. subconjuntos preprocesados**

Usen estos David L subconjuntos ya
preprocesados y divididos:

- `TA1C_dataset_detection_train_split.csv` → contiene el conjunto de entrenamiento (75%).
- `TA1C_dataset_detection_dev_split.csv` → contiene el conjunto de desarrollo (25%).

Ambos incluyen únicamente las columnas `Teaser Text` y `Tag Value` y están listos para ser
usados en la etapa de tokenización y entrenamiento del LLM.


In [13]:
df_train.to_csv("TA1C_dataset_detection_train_split.csv", index=False)
df_dev.to_csv("TA1C_dataset_detection_dev_split.csv", index=False)

print("Archivos guardados:")
print(" - TA1C_dataset_detection_train_split.csv")
print(" - TA1C_dataset_detection_dev_split.csv")

Archivos guardados:
 - TA1C_dataset_detection_train_split.csv
 - TA1C_dataset_detection_dev_split.csv
